In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

current_dir = os.getcwd()
print(current_dir)

/home/kacper/semestr_6/ZaawansowaneAlgorytmyWizyjne/lab3


In [7]:
pedastrian = cv2.imread (f"{current_dir}/pedestrian/input/in000300.jpg")
yy, xx, _ = pedastrian.shape
N = 60
iN = 0



In [8]:
BUF = np . zeros (( yy ,xx ,N) , np . uint8 )

In [9]:
def median_and_erote_dilate(I: np.array) -> np.array:
    I_copy = I.copy()
    I_copy = cv2.medianBlur(I_copy, 5)
    kernel = np.ones((5, 5), np.uint8)
    I_copy = cv2.dilate(I_copy, kernel)
    I_copy = cv2.erode(I_copy, kernel)
    return I_copy

def calculate_true_positive(I: np.array, I_copy: np.array) -> int:
    TP = 0
    TP_M = lambda x,y: np.logical_and((x == 255), (y == 255))
    TP_S = np.sum(TP_M(I, I_copy)) # sum of the elements in the matrix
    TP += TP_S
    return TP

def calculate_false_negative(I: np.array, I_copy: np.array) -> int:
    FN = 0
    FN_M = lambda x,y: np.logical_and((x == 255), (y == 0))
    FN_S = np.sum(FN_M(I, I_copy)) # sum of the elements in the matrix
    FN += FN_S
    return FN

def calculate_true_negative(I: np.array, I_copy: np.array) -> int:
    TN = 0
    TN_M = lambda x,y: np.logical_and((x == 0), (y == 0))
    TN_S = np.sum(TN_M(I, I_copy)) # sum of the elements in the matrix
    TN += TN_S
    return TN

def calculate_false_positive(I: np.array, I_copy: np.array) -> int:
    FP = 0
    FP_M = lambda x,y: np.logical_and((x == 0), (y == 255))
    FP_S = np.sum(FP_M(I, I_copy)) # sum of the elements in the matrix
    FP += FP_S
    return FP

In [24]:
# 3.1 srednia
import numpy.ma as ma

true_positive_array = []
false_negative_array = []
true_negative_array = []
false_positive_array = [] 
BUF = np.zeros((yy, xx, N), np.uint8)

for i in range(300, 999):
    I = cv2.imread(f"{current_dir}/pedestrian/input/in%06d.jpg" % i )
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY).astype(np.uint8)
    BUF[:,:,iN] = IG.astype(np.uint8)

    BUF_masked = ma.masked_equal(BUF, 0)

    # Calculate the mean only from values different than zeros
    mean = BUF_masked.mean(axis=2)

    median = ma.median(BUF_masked, axis=2)
    iN += 1
    if iN == N:
        iN = 0

    I_diff = cv2.absdiff(IG, mean.astype(np.uint8))
    I_diff_binarized = cv2.threshold(I_diff, 30, 255, cv2.THRESH_BINARY)[1]
    I_diff_binarized_median = cv2.medianBlur(I_diff_binarized, 5)
    I_diff_binarized_dilated = cv2.dilate(I_diff_binarized_median, np.ones((5,5), np.uint8), iterations=1)
    I_diff_binarized_eroded = cv2.erode(I_diff_binarized_dilated, np.ones((5,5), np.uint8), iterations=1)
    cv2.imshow("I_diff_binarized_eroded", I_diff_binarized_eroded)
    cv2.waitKey(1)

    groundtruth = cv2.imread (f"{current_dir}/pedestrian/groundtruth/gt%06d.png" % i, cv2.IMREAD_GRAYSCALE )
    groundtruth_binarized = (groundtruth > 150).astype('uint8') * 255
    cv2.imshow("groundtruth_binarized", groundtruth_binarized)
    cv2.waitKey(1)
    true_positive_array.append(calculate_true_positive(groundtruth_binarized, I_diff_binarized_eroded))
    false_negative_array.append(calculate_false_negative(groundtruth_binarized, I_diff_binarized_eroded))
    true_negative_array.append(calculate_true_negative(groundtruth_binarized, I_diff_binarized_eroded))
    false_positive_array.append(calculate_false_positive(groundtruth_binarized, I_diff_binarized_eroded))

precision = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_negative_array))
print("Preicison: ", precision)
recall = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_positive_array))
print("Recall: ", recall)
f1 = 2 * ((precision * recall) / (precision + recall))
print("F1: ", f1)
    

cv2.destroyAllWindows()

Preicison:  0.6055530452616311
Recall:  0.8263851737556103
F1:  0.6989408507724824


In [25]:
# 3.1 mediana
true_positive_array = []
false_negative_array = []
true_negative_array = []
false_positive_array = [] 
BUF = np . zeros (( yy ,xx ,N) , np . uint8 )
for i in range(300, 1100):
    I = cv2.imread(f"{current_dir}/pedestrian/input/in%06d.jpg" % i )
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY).astype(np.uint8)
    BUF[:,:,iN] = IG.astype(np.uint8)
    
    # Create a masked array where the condition is that the values are not equal to zero
    BUF_masked = ma.masked_equal(BUF, 0)

    # Calculate the mean only from values different than zeros
    mean = BUF_masked.mean(axis=2)

    median = ma.median(BUF_masked, axis=2)
    iN += 1
    if iN == N:
        iN = 0

    I_diff = cv2.absdiff(IG, median.astype(np.uint8))
    I_diff_binarized = cv2.threshold(I_diff, 30, 255, cv2.THRESH_BINARY)[1]
    I_diff_binarized_median = cv2.medianBlur(I_diff_binarized, 5)
    I_diff_binarized_dilated = cv2.dilate(I_diff_binarized_median, np.ones((5,5), np.uint8), iterations=1)
    I_diff_binarized_eroded = cv2.erode(I_diff_binarized_dilated, np.ones((5,5), np.uint8), iterations=1)

    cv2.imshow("I_diff_binarized_eroded", I_diff_binarized_eroded)
    if cv2.waitKey(1) == ord('q'):
        break

    groundtruth = cv2.imread (f"{current_dir}/pedestrian/groundtruth/gt%06d.png" % i, cv2.IMREAD_GRAYSCALE )
    groundtruth_binarized = (groundtruth > 150).astype('uint8') * 255
    cv2.imshow("groundtruth_binarized", groundtruth_binarized)
    if cv2.waitKey(1) == ord('q'):
        break
    true_positive_array.append(calculate_true_positive(groundtruth_binarized, I_diff_binarized_eroded))
    false_negative_array.append(calculate_false_negative(groundtruth_binarized, I_diff_binarized_eroded))
    true_negative_array.append(calculate_true_negative(groundtruth_binarized, I_diff_binarized_eroded))
    false_positive_array.append(calculate_false_positive(groundtruth_binarized, I_diff_binarized_eroded))

precision = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_negative_array))
print("Preicison: ", precision)
recall = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_positive_array))
print("Recall: ", recall)
f1 = 2 * ((precision * recall) / (precision + recall))
print("F1: ", f1)
    

cv2.destroyAllWindows()

Preicison:  0.5716660342927349
Recall:  0.9838920330866348
F1:  0.7231586766470487


In [26]:
#3.2 srednia
true_positive_array = []
false_negative_array = []
true_negative_array = []
false_positive_array = [] 
BG = cv2.imread(f"{current_dir}/pedestrian/input/in000300.jpg", cv2.IMREAD_GRAYSCALE).astype(int)
for i in range(301, 1100):
    I = cv2.imread(f"{current_dir}/pedestrian/input/in%06d.jpg" % i )
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY).astype(int)

    alfa = 0.03
    BG = (alfa * IG + (1 - alfa) * BG).astype(np.float64)


    I_diff = cv2.absdiff(IG.astype(np.float64), BG.astype(np.float64)).astype(np.uint8)
    I_diff_binarized = cv2.threshold(I_diff, 35, 255, cv2.THRESH_BINARY)[1]
    I_diff_binarized_median = cv2.medianBlur(I_diff_binarized, 5).astype(np.uint8)
    I_diff_binarized_dilated = cv2.dilate(I_diff_binarized_median, np.ones((5, 5), np.uint8), iterations=1).astype(np.uint8)
    I_diff_binarized_eroded = cv2.erode(I_diff_binarized_dilated, np.ones((5,5), np.uint8), iterations=1).astype(np.uint8)
    cv2.imshow("I_diff_binarized_eroded", I_diff_binarized_eroded)
    cv2.waitKey(1)
   

    groundtruth = cv2.imread (f"{current_dir}/pedestrian/groundtruth/gt%06d.png" % i, cv2.IMREAD_GRAYSCALE )
    groundtruth_binarized = (groundtruth > 150).astype('uint8') * 255
    cv2.imshow("groundtruth_binarized", groundtruth_binarized)
    cv2.waitKey(1)
    true_positive_array.append(calculate_true_positive(groundtruth_binarized, I_diff_binarized_eroded))
    false_negative_array.append(calculate_false_negative(groundtruth_binarized, I_diff_binarized_eroded))
    true_negative_array.append(calculate_true_negative(groundtruth_binarized, I_diff_binarized_eroded))
    false_positive_array.append(calculate_false_positive(groundtruth_binarized, I_diff_binarized_eroded))

precision = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_negative_array))
print("Preicison: ", precision)
recall = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_positive_array))
print("Recall: ", recall)
f1 = 2 * ((precision * recall) / (precision + recall))
print("F1: ", f1)
    

cv2.destroyAllWindows()

Preicison:  0.5637726135845607
Recall:  0.9232080956287702
F1:  0.7000486794888013


In [27]:
#3.2 mediana

true_positive_array = []
false_negative_array = []
true_negative_array = []
false_positive_array = [] 
BG = cv2.imread(f"{current_dir}/pedestrian/input/in000300.jpg", cv2.IMREAD_GRAYSCALE).astype(int)
for i in range(301, 1100):
    I = cv2.imread(f"{current_dir}/pedestrian/input/in%06d.jpg" % i )
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY).astype(int)

    alfa = 0.03
    
    BG[np.where(BG > IG)] -= 1
    BG[np.where(BG < IG)] += 1
    

    I_diff = cv2.absdiff(IG.astype(np.uint8), BG.astype(np.uint8))
    I_diff_binarized = cv2.threshold(I_diff, 35, 255, cv2.THRESH_BINARY)[1]
    I_diff_binarized_median = cv2.medianBlur(I_diff_binarized, 5).astype(np.uint8)
    I_diff_binarized_dilated = cv2.dilate(I_diff_binarized_median, np.ones((5, 5), np.uint8), iterations=1).astype(np.uint8)
    I_diff_binarized_eroded = cv2.erode(I_diff_binarized_dilated, np.ones((5,5), np.uint8), iterations=1).astype(np.uint8)
    cv2.imshow("I_diff_binarized_eroded", I_diff_binarized_eroded)
    cv2.waitKey(1)
   

    groundtruth = cv2.imread (f"{current_dir}/pedestrian/groundtruth/gt%06d.png" % i, cv2.IMREAD_GRAYSCALE )
    groundtruth_binarized = (groundtruth > 150).astype('uint8') * 255
    cv2.imshow("groundtruth_binarized", groundtruth_binarized)
    cv2.waitKey(1)
    true_positive_array.append(calculate_true_positive(groundtruth_binarized, I_diff_binarized_eroded))
    false_negative_array.append(calculate_false_negative(groundtruth_binarized, I_diff_binarized_eroded))
    true_negative_array.append(calculate_true_negative(groundtruth_binarized, I_diff_binarized_eroded))
    false_positive_array.append(calculate_false_positive(groundtruth_binarized, I_diff_binarized_eroded))

precision = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_negative_array))
print("Preicison: ", precision)
recall = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_positive_array))
print("Recall: ", recall)
f1 = 2 * ((precision * recall) / (precision + recall))
print("F1: ", f1)
    

cv2.destroyAllWindows()

Preicison:  0.5703946049147168
Recall:  0.984319799535558
F1:  0.7222557423520354


In [28]:

# 3.3 polityka konserwatywna - aktualizacja tla

true_positive_array = []
false_negative_array = []
true_negative_array = []
false_positive_array = [] 
BG = cv2.imread(f"{current_dir}/pedestrian/input/in000300.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float64)
mask = np.zeros_like(BG)

for i in range(301, 1100):
    I = cv2.imread(f"{current_dir}/pedestrian/input/in%06d.jpg" % i )
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY).astype(int)
    alfa = 0.03
    BG = np.where(mask == 0, BG, alfa * IG + (1 - alfa) * BG)

    I_diff = cv2.absdiff(IG.astype(np.uint8), BG.astype(np.uint8)).astype(np.uint8)
    I_diff_binarized = cv2.threshold(I_diff, 35, 255, cv2.THRESH_BINARY)[1]
    I_diff_binarized_median = cv2.medianBlur(I_diff_binarized, 5).astype(np.uint8)
    I_diff_binarized_dilated = cv2.dilate(I_diff_binarized_median, np.ones((5, 5), np.uint8), iterations=1).astype(np.uint8)
    I_diff_binarized_eroded = cv2.erode(I_diff_binarized_dilated, np.ones((5,5), np.uint8), iterations=1).astype(np.uint8)
    cv2.imshow("I_diff_binarized_eroded", I_diff_binarized_eroded)
    cv2.waitKey(1)


    groundtruth = cv2.imread (f"{current_dir}/pedestrian/groundtruth/gt%06d.png" % i, cv2.IMREAD_GRAYSCALE )
    groundtruth_binarized = (groundtruth > 150).astype('uint8') * 255
    cv2.imshow("groundtruth_binarized", groundtruth_binarized)
    
    true_positive_array.append(calculate_true_positive(groundtruth_binarized, I_diff_binarized_eroded))
    false_negative_array.append(calculate_false_negative(groundtruth_binarized, I_diff_binarized_eroded))
    true_negative_array.append(calculate_true_negative(groundtruth_binarized, I_diff_binarized_eroded))
    false_positive_array.append(calculate_false_positive(groundtruth_binarized, I_diff_binarized_eroded))

precision = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_negative_array))
print("Preicison: ", precision)
recall = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_positive_array))
print("Recall: ", recall)
f1 = 2 * ((precision * recall) / (precision + recall))
print("F1: ", f1)
    

cv2.destroyAllWindows()

Preicison:  0.6039842851827947
Recall:  0.9838461498064794
F1:  0.7484774197878694


In [29]:
# 3.4 GMM / MOG

true_positive_array = []
false_negative_array = []
true_negative_array = []
false_positive_array = []
history = 64
threshold = 64
detect_shadow = False
fgbg = cv2.createBackgroundSubtractorMOG2(history=history, varThreshold=threshold, detectShadows=detect_shadow)
for i in range(301, 1100):
    I = cv2.imread(f"{current_dir}/pedestrian/input/in%06d.jpg" % i )
    fgmask = fgbg.apply(I, learningRate=0.001)
    I_diff_binarized_eroded = fgmask
    groundtruth = cv2.imread (f"{current_dir}/pedestrian/groundtruth/gt%06d.png" % i, cv2.IMREAD_GRAYSCALE )
    groundtruth_binarized = (groundtruth > 150).astype('uint8') * 255
    cv2.imshow("I_diff_binarized_eroded", I_diff_binarized_eroded)
    cv2.waitKey(1)
    cv2.imshow("groundtruth_binarized", groundtruth_binarized)
    cv2.waitKey(1)
    true_positive_array.append(calculate_true_positive(groundtruth_binarized, I_diff_binarized_eroded))
    false_negative_array.append(calculate_false_negative(groundtruth_binarized, I_diff_binarized_eroded))
    true_negative_array.append(calculate_true_negative(groundtruth_binarized, I_diff_binarized_eroded))
    false_positive_array.append(calculate_false_positive(groundtruth_binarized, I_diff_binarized_eroded))
    
precision = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_negative_array))
recall = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_positive_array))
f1 = 2 * ((precision * recall) / (precision + recall))
print("Preicison: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
cv2.destroyAllWindows()

Preicison:  0.7810700028951062
Recall:  0.8502020654329843
F1:  0.814171152197532


In [30]:
# 3.5 KNN

true_positive_array = []
false_negative_array = []
true_negative_array = []
false_positive_array = []
history = 8
threshold = 4
detect_shadow = False
fgbg = cv2.createBackgroundSubtractorKNN()
for i in range(301, 1100):
    I = cv2.imread(f"{current_dir}/pedestrian/input/in%06d.jpg" % i )
    fgmask = fgbg.apply(I)
    I_diff_binarized_eroded = fgmask
    groundtruth = cv2.imread (f"{current_dir}/pedestrian/groundtruth/gt%06d.png" % i, cv2.IMREAD_GRAYSCALE )
    groundtruth_binarized = (groundtruth > 150).astype('uint8') * 255
    cv2.imshow("I_diff_binarized_eroded", I_diff_binarized_eroded)
    cv2.waitKey(1)
    cv2.imshow("groundtruth_binarized", groundtruth_binarized)
    cv2.waitKey(1)
    true_positive_array.append(calculate_true_positive(groundtruth_binarized, I_diff_binarized_eroded))
    false_negative_array.append(calculate_false_negative(groundtruth_binarized, I_diff_binarized_eroded))
    true_negative_array.append(calculate_true_negative(groundtruth_binarized, I_diff_binarized_eroded))
    false_positive_array.append(calculate_false_positive(groundtruth_binarized, I_diff_binarized_eroded))
    
precision = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_negative_array))
recall = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_positive_array))
f1 = 2 * ((precision * recall) / (precision + recall))
print("Preicison: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
cv2.destroyAllWindows()

Preicison:  0.7752261456215804
Recall:  0.6705224563652923
F1:  0.7190828871443027


: 